In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

from fastmri_recon.data.test_generators import RandomShapeGenerator
from fastmri_recon.helpers.adversarial_training import compile_models, adversarial_training_loop
from fastmri_recon.helpers.image_tboard_cback import TensorBoardImage
from fastmri_recon.helpers.keras_utils import wasserstein_loss
from fastmri_recon.models.discriminator import discriminator_model, generator_containing_discriminator_multiple_outputs
from fastmri_recon.models.unet import unet
from fastmri_recon.helpers.utils import keras_ssim, keras_psnr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
}

im_size = 64

def generator_model():
    model = unet(input_size=(im_size, im_size, 1), **run_params, compile=False)
    model.name = 'Reconstructor'
    return model

In [130]:
g = generator_model()
g_opt = Adam(lr=1e-3, clipnorm=1.)
g.compile(optimizer=g_opt, loss='mse', metrics=[keras_psnr, keras_ssim])

In [135]:
n_batches_train = 1000
n_batches_val = 1
AF = 2
train_gen = RandomShapeGenerator(af=AF, n_shapes=100, size=im_size, batch_size=1).flow_z_filled_random_shapes()
val_gen = RandomShapeGenerator(af=AF, n_shapes=100, size=im_size, batch_size=1).flow_z_filled_random_shapes()

epoch_num = 100


run_id = f'gen_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

gen_af2_1572957673


In [136]:
log_dir = op.join('logs2', run_id)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    write_graph=True, 
    update_freq=50,
)
chkpt_cback = ModelCheckpoint(chkpt_path, period=epoch_num)
selected_slice = 0
data = next(val_gen)
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=data[1][selected_slice:selected_slice+1],
    # NOTE: for cross-domain slice has to be on kspace and mask
    model_input=data[0][selected_slice:selected_slice+1],
)

In [126]:
aliased = []
og = []
for i in range(150):
    x,image = next(train_gen) 
    aliased.append(x)
    og.append(image)
    

In [127]:
import numpy
a = np.squeeze(aliased, axis=1)
o = np.squeeze(og, axis=1)

In [128]:
g.fit(x=a, y=o, batch_size=1, epochs=100, callbacks=[tqdm_cb, tboard_cback, tboard_image_cback])

Epoch 1/100


150/150 [==============================] - 12s 83ms/step - loss: 0.0628 - keras_psnr: 12.4713 - keras_ssim: 0.3656
Epoch 2/100


150/150 [==============================] - 14s 93ms/step - loss: 0.0454 - keras_psnr: 13.2852 - keras_ssim: 0.4259
Epoch 3/100


150/150 [==============================] - 18s 117ms/step - loss: 0.0448 - keras_psnr: 13.3466 - keras_ssim: 0.4351
Epoch 4/100


150/150 [==============================] - 18s 119ms/step - loss: 0.0442 - keras_psnr: 13.4717 - keras_ssim: 0.4374
Epoch 5/100


...

150/150 [==============================] - 14s 93ms/step - loss: 0.0039 - keras_psnr: 24.0229 - keras_ssim: 0.8082
Epoch 97/100


150/150 [==============================] - 11s 73ms/step - loss: 0.0035 - keras_psnr: 24.5497 - keras_ssim: 0.8176
Epoch 98/100


150/150 [==============================] - 11s 74ms/step - loss: 0.0032 - keras_psnr: 24.8822 - keras_ssim: 0.8270
Epoch 99/100


150/150 [==============================] - 93s 619ms/step - loss: 0.0032 - keras_psnr: 24.9427 - keras_ssim: 0.8274
Epoch 100/100


150/150 [==============================] - 14s 92ms/step - loss: 0.0032 - keras_psnr: 24.8402 - keras_ssim: 0.8288


In [137]:
aliased = []
og = []
for i in range(500):
    x,image = next(train_gen) 
    aliased.append(x)
    og.append(image)

a = np.squeeze(aliased, axis=1)
o = np.squeeze(og, axis=1)
g.fit(x=a, y=o, batch_size=1, epochs=1, callbacks=[tqdm_cb, tboard_cback, tboard_image_cback])

Epoch 1/1


500/500 [==============================] - 39s 78ms/step - loss: 0.0441 - keras_psnr: 13.4544 - keras_ssim: 0.4401
